In [ ]:
import spacy
import pandas as pd
from textblob import TextBlob
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
import string
import json
from ast import literal_eval 
import requests
import geopandas
from tqdm.auto import tqdm
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor


nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
spacy.cli.download("en_core_web_sm")
spacy.cli.download("fr_core_news_sm")

plt.style.use('bmh')

# This is where you load the exported data
file_path = 'all_languages/items.json'

with open(file_path, 'r') as file:
    data = json.load(file)

# # Just
# def collect_labels(obj, prefix='', label_dict=None):
#     if label_dict is None:
#         label_dict = {}

#     if isinstance(obj, dict):
#         for key, value in obj.items():
#             label = prefix + '.' + key if prefix else key
#             label_dict[label] = type(value).__name__
#             collect_labels(value, label, label_dict)
#     elif isinstance(obj, list):
#         for item in obj:
#             collect_labels(item, prefix, label_dict)

#     return label_dict

# label_hierarchy = collect_labels(data)

# Reading from the exported JSON
def normalize_data(entry):
    base_info = {
        'type': entry.get('@type'),
        'id': entry.get('@id'),
        'graph': entry.get('@graph'),
        'label': entry.get('label'),
        'source_id': entry.get('source', {}).get('@id'),
        'source_label': entry.get('source', {}).get('label'),
        'source_url': entry.get('source', {}).get('url'),
        'source_date': entry.get('source', {}).get('date'),
        'source_language': entry.get('source', {}).get('language'),
        'source_genre': [genre.get('label') for genre in entry.get('source', {}).get('genre', [])],
        'relevantExcerpt': entry.get('relevantExcerpt'),
        'adjective': entry.get('adjective'),
        'emotion': [emotion.get('label') for emotion in entry.get('emotion', [])],
        'license': entry.get('license'),
        'time': [{'id': time.get('@id'), 'label': time.get('label'), 'begin': time.get('begin'), 'end': time.get('end')} for time in entry.get('time', [])],
        'place': [{'id': place.get('@id'), 'label': place.get('label')} for place in entry.get('place', [])],
        'smellSource': [{'id': source.get('@id'), 'label': source.get('label')} for source in entry.get('smellSource', [])],
        'carrier': [{'id': carrier.get('@id'), 'label': carrier.get('label'), 'exemplifies': carrier.get('exemplifies')} for carrier in entry.get('carrier', [])],
    }

    rows = []
    for excerpt in entry.get('source', {}).get('excerpts', []):
        excerpt_data = {
            'excerpt_id': excerpt.get('@id'),
            'excerpt_value': excerpt.get('value'),
            'words': excerpt.get('words', [])
        }

        if not isinstance(excerpt_data['words'], list):
            excerpt_data['words'] = [excerpt_data['words']]

        row = {**base_info, **excerpt_data}
        rows.append(row)

    return rows

rows = []
for entry in data:
    rows.extend(normalize_data(entry))

items_pd = pd.DataFrame(rows)


# Filtering by languages (en, fr)
filtered_df = items_pd[items_pd['source_language'].isin(['en', 'fr'])]
filtered_df = filtered_df.reset_index(drop=True)

# Load language models
en_nlp = spacy.load("en_core_web_sm")
fr_nlp = spacy.load("fr_core_news_sm")

# Initialize lemmatizers and stopwords
en_lemmatizer = WordNetLemmatizer()
fr_lemmatizer = lambda text: " ".join([token.lemma_ for token in fr_nlp(text)])
en_stopwords = set(stopwords.words('english'))
fr_stopwords = set(stopwords.words('french'))


##############
# Cleaning function, for both languages
# This is for the texts in excerpt_value and source_label
##############
def cleaning(text, language):
    # Lowercasing
    text = text.lower()
    
    # No punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    if language == 'en':
        # English preprocessing
        # No stopwords
        text = ' '.join([word for word in text.split() if word not in en_stopwords])
        
        # Lemmatize text
        word_tokens = word_tokenize(text)
        text = ' '.join([en_lemmatizer.lemmatize(w) for w in word_tokens])
    
    elif language == 'fr':
        # French preprocessing
        # No stopwords
        text = ' '.join([word for word in text.split() if word not in fr_stopwords])
        
        # Lemmatize text
        text = fr_lemmatizer(text)
    
    return text

filtered_df['excerpt_value_cleaned'] = filtered_df.apply(lambda row: cleaning(row['excerpt_value'], row['source_language']), axis=1)
filtered_df['source_label_cleaned'] = filtered_df.apply(lambda row: cleaning(row['source_label'], row['source_language']), axis=1)


############
# This is for cleaning the year/date column, change varying formats to normal. If not possible, we get the median of the items in that 
# specific graph and then use as the date in the row.
############

# Preprocess 'source_date' to handle varying formats
def preprocess_date(date):
    if pd.isna(date):
        return np.nan
    if '/' in date:
        return date.split('/')[0]  # Take the first year in case of '1855/1856'
    if 'XX' in date:
        return date.replace('XX', '50')  # Replace 'XX' with '50' for '18XX'
    return date

# Apply preprocessing to 'source_date'
filtered_df['source_date'] = filtered_df['source_date'].apply(preprocess_date)

# Convert 'source_date' to numeric, coerce errors to NaN
filtered_df['source_date'] = pd.to_numeric(filtered_df['source_date'], errors='coerce')

# Calculate the median year for each graph group
median_years = filtered_df.groupby('graph')['source_date'].median()

# A function to fill missing values with the median of the respective graph
def fill_with_median(row):
    if pd.isna(row['source_date']):
        return median_years.get(row['graph'], np.nan)  # Get the median year, default to NaN if no median exists
    else:
        return row['source_date']

# Apply the function to fill missing 'source_date' values
filtered_df['source_date'] = filtered_df.apply(fill_with_median, axis=1)

############
# Extracting the last part of the link
############

# Define a function to extract the last part of a URL
def extract_last_part(url):
    # Check if the entry is a string and contains a slash
    if isinstance(url, str) and '/' in url:
        return url.split('/')[-1]  # Split by '/' and return the last element
    return url  # Return the original value if it's not a string or doesn't contain '/'

# Columns to clean
columns_to_clean = ['type', 'id', 'graph', 'source_id', 'relevantExcerpt', 'excerpt_id']

# Apply the function to each specified column
for column in columns_to_clean:
    filtered_df[column] = filtered_df[column].apply(extract_last_part)

############
# Changing the place from exact areas to countries to simplify as it is not needed for this project
# We will be using geonames dataset to transform them from the exact place to countries names and codes
############

# Function to extract the geoname ID from a URL
def extract_geoname_id(url):
    if pd.isna(url):
        return None
    return url.split('/')[-2]

# Function to match the country code using a dictionary for fast lookup
def match_country(geoname_id, geo_dict):
    return geo_dict.get(geoname_id)

# Function to process each place array and extract unique country codes
def extract_and_match_countries(place_array, geo_dict):
    country_codes = set()
    for place in place_array:
        geoname_id = extract_geoname_id(place['id'])
        country_code = match_country(geoname_id, geo_dict)
        if country_code:
            country_codes.add(country_code)
    return list(country_codes)

# Process each row to extract country codes
def process_row(place_array, geo_dict):
    return extract_and_match_countries(place_array, geo_dict)

# Load GeoNames data and convert to dictionary for faster access
file_path = 'allCountries.txt'
cols = ['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 
        'feature class', 'feature code', 'country code', 'cc2', 'admin1', 'admin2', 
        'admin3', 'admin4', 'population', 'elevation', 'dem', 'timezone', 'modification date']
geo_data = pd.read_csv(file_path, delimiter='\t', names=cols, low_memory=False, encoding='utf-8', na_values='\\N', dtype={'geonameid': str})
geo_dict = pd.Series(geo_data['country code'].values, index=geo_data['geonameid']).to_dict()

# Use ThreadPoolExecutor to parallelize row processing
with ThreadPoolExecutor() as executor:
    countries_results = list(executor.map(lambda x: process_row(x, geo_dict), filtered_df['place']))
filtered_df['country_codes'] = countries_results

middle_east_countries = {
    'YE': 'Yemen',
    'IQ': 'Iraq',
    'SD': 'Sudan',
    'SA': 'Saudi Arabia',
    'PS': 'Palestine',  # Ensure Palestine is here
    'JO': 'Jordan',
    'OM': 'Oman',
    'EG': 'Egypt',
    'LB': 'Lebanon',
    # 'IL': 'Israel',  # This can be removed or handled differently
    'AE': 'United Arab Emirates',
    'SY': 'Syria',
    'IR': 'Iran'
}

# Function to convert country codes to country names using the dictionary, handling "IL" specifically
def codes_to_names(codes):
    result = []
    for code in codes:
        if code == 'IL':  # Convert "IL" to "PS"
            result.append('Palestine')
        elif code in middle_east_countries:
            result.append(middle_east_countries[code])
    return result

# Apply this function to each row in the 'country_codes' column
filtered_df['country_names'] = filtered_df['country_codes'].apply(codes_to_names)

##############
# Merging and cleaning rows
#
##############

def merge_complex_lists(lists):
    result = []
    seen = set()  # to track seen values for simple items or dict items
    for sublist in lists:
        if sublist is None:
            continue  # Skip None values
        for item in sublist:
            if isinstance(item, dict):
                # Convert dict to a tuple of its items to make it hashable
                item_tuple = tuple(sorted(item.items()))
                if item_tuple not in seen:
                    seen.add(item_tuple)
                    result.append(item)
            else:
                if item not in seen:
                    seen.add(item)
                    result.append(item)
    return result

def combine_rows(group):
    first_row = group.iloc[0].copy()  # Base row for accumulation
    for column in group.columns:
        if column == 'label':
            # Merge 'label' ensuring no duplicate strings and handling lists
            combined_labels = set()
            for entry in group[column]:
                if isinstance(entry, list):
                    combined_labels.update(entry)
                elif isinstance(entry, str):
                    combined_labels.add(entry)
            first_row[column] = list(combined_labels)
        elif column in ['smellSource', 'country_names', 'carrier', 'emotion']:
            # Handle lists, possibly containing dicts
            all_entries = group[column].tolist()
            first_row[column] = merge_complex_lists(all_entries)
        else:
            # For other columns, take the first non-null value
            non_null_values = group[column].dropna()
            if not non_null_values.empty:
                first_row[column] = non_null_values.iloc[0]
    return first_row

# Example DataFrame setup and applying the functions
# Assuming filtered_df is previously defined with columns like 'excerpt_value', 'source_date', etc.
# Apply the combination logic after normalizing data for processing
filtered_df = filtered_df.groupby(['excerpt_value', 'source_date']).apply(combine_rows).reset_index(drop=True)

############
# Using the existing annotated emotions, we will clean it to map to the primary Plutchik emotion
############

primary_emotions_map = {
    'calmness': 'joy', 'serenity': 'joy', 'joy': 'joy', 'ecstasy': 'joy',
    'excitement': 'joy', 'relief': 'joy', 'despair': 'sadness', 'pensiveness': 'sadness',
    'sadness': 'sadness', 'disappointment': 'sadness', 'grief': 'sadness', 'desire':'anticipation',
    'embarrassment': 'fear', 'nostalgia': 'sadness', 'pain': 'sadness', 'greed':'anger',
    'approval': 'trust', 'acceptance': 'trust', 'trust': 'trust', 'admiration': 'joy', 'courage':'anticipation',
    'faith': 'trust', 'indifference': 'disgust', 'boredom': 'sadness', 'disgust': 'disgust',
    'loathing': 'disgust', 'nervousness': 'fear', 'apprehension': 'fear', 'pride': 'joy',
    'fear': 'fear', 'terror': 'fear', 'annoyance': 'anger', 'frustration': 'anger', 'love':'joy',
    'anger': 'anger', 'rage': 'anger', 'envy': 'anger', 'surprise': 'surprise', 'gratitude':'joy',
    'amazement': 'surprise', 'curiosity': 'anticipation', 'interest': 'anticipation', 'guilt':'sadness',
    'anticipation': 'anticipation', 'vigilance': 'anticipation', 'doubt': 'fear', 'amusement':'joy',
    'optimism': 'anticipation', 'disapproval': 'disgust'
}

# Apply the mapping with improved checks
filtered_df['plutchik_emotion'] = filtered_df['emotion'].apply(
    lambda x: primary_emotions_map[x[0]] if x and isinstance(x, list) and len(x) > 0 else None
)

filtered_df.drop(columns=['source_url', 'place', 'time', 'type', 'excerpt_value', 'source_label'], inplace=True)

filtered_df.to_csv(f"data_{pd.Timestamp.now().strftime('%Y-%m-%d_%H-%M')}.csv")

filtered_df.head()